### Project Dashboard developed in Bokeh

run in terminal from root directory<p>

```
bokeh serve --show project_dashboard.ipynb
```

In [20]:
import json
from bokeh.io import show, output_file, output_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,Range1d,
                          LinearColorMapper, Slider, Select, RadioButtonGroup, Column, Text, Plot, LinearAxis, Grid, TapTool)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

from chart_constants import FONT_PROPS_LG, PLOT_FORMATS

import geopandas as gpd
import pandas as pd
import numpy as np
import os

import logging
  

In [10]:
# loading chemichal data dataset 
chem_data = pd.read_csv(r'assets/ChemDataForJeffOlson.csv', parse_dates=['VisitDate'])
chem_data['year'] = chem_data.VisitDate.dt.year

seasons = {
    '0':'winter',
    '1':'spring',
    '2':'summer',
    '3':'fall'
}

chem_data['season'] =  (chem_data.VisitDate.dt.month - 1) //3
chem_data['season'] = chem_data.season.apply(lambda x: seasons[str(x)])
chem_data['date'] = chem_data['VisitDate'].apply(lambda x:x.strftime("%Y-%m-01"))

In [11]:
df = chem_data.groupby(['LakeID', 'year', 'CharacteristicID'])['Result'].agg(np.mean)
df = df.reset_index()

In [12]:
gdf = gpd.read_file(r'assets/Lakes_Inventory.geojson')

In [13]:
def column_data(selectedYear,CharacteristicID):
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    result_min = df[df.CharacteristicID == CharacteristicID].Result.min()
    result_max = df[df.CharacteristicID == CharacteristicID].Result.max()
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)
    
    return data, result_min, result_max


def json_data(selectedYear,CharacteristicID):
    
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    result_min = df[df.CharacteristicID == CharacteristicID].Result.min()
    result_max = df[df.CharacteristicID == CharacteristicID].Result.max()
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)


    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)


    return json_data, result_min, result_max

def time_series_data(CharacteristicID, index):
 
    LakeID = gdf.iloc[index].LAKEID
    df_Lake = df[(df['CharacteristicID'] == CharacteristicID) & (df['LakeID'] == LakeID)]

    
    return df_Lake

def time_series_LakeID(index):
    return gdf.iloc[index].LAKEID

In [14]:
#Input GeoJSON source that contains features for plotting.
geosource=GeoJSONDataSource(geojson=json_data(1980, "TP")[0])

In [21]:
source=ColumnDataSource(time_series_data("TP", 0))

#Define a sequential multi-hue color palette.
palette = brewer["YlGnBu"][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = "#d9d9d9")

#Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 20, height = 550,
                    border_line_color=None, location = (0,0), orientation = "vertical")

#Add hover tool
hover = HoverTool(tooltips = [ ("Lake","@LakeID"),("Result", "@Result")])

tap = TapTool(behavior='select', gesture='tap')


# Create a text object to display leake health metric
                                                            
xdr = Range1d(0, 220)                                                       
ydr = Range1d(0, 120)                                                       

plot = Plot(                                                                
    x_range=xdr,                                                            
    y_range=ydr,                                                            
    title="",                                                               
    plot_width=500,                                                         
    plot_height=200,                                                        
    min_border=0,
    **PLOT_FORMATS
                                                           
)

# Add the writing                                                           
metric_title = Text(x=0, y=0, text=['Lake Health metric'], **FONT_PROPS_LG)
metric_value = Text(x=150,y=0, text=['50'], **FONT_PROPS_LG)
plot.add_glyph(metric_title)
plot.add_glyph(metric_value)

wedge = figure(plot_height = 200,
           plot_width = 200, 
           toolbar_location = None)

wedge.xgrid.visible = False
wedge.ygrid.visible = False
wedge.yaxis.visible = False
wedge.xaxis.visible = False
wedge.xaxis.major_tick_line_color = None  
wedge.xaxis.minor_tick_line_color = None
wedge.yaxis.minor_tick_line_color = None
wedge.yaxis.major_tick_line_color = None
wedge.yaxis.major_label_text_font_size = '0pt'  
wedge.xaxis.major_label_text_font_size = '0pt'
wedge.outline_line_color=None

wedge.annular_wedge(x=[2], y=[2], inner_radius=0.50, outer_radius=1,
                start_angle=0.4, end_angle=4.8, color="green", alpha=0.6)


# Create a figure object for line chart

time_series = figure(title = 'Chemichal data time-series', 
           plot_height = 200,
           plot_width = 500, 
           toolbar_location = None)
time_series.xgrid.visible = False
time_series.ygrid.visible = False
time_series.xaxis.major_tick_line_color = 'grey'  
time_series.xaxis.minor_tick_line_color = None
time_series.yaxis.minor_tick_line_color = None
time_series.yaxis.major_tick_line_color = 'grey'
time_series.yaxis.major_label_text_font_size = '10pt'  
time_series.xaxis.major_label_text_font_size = '10pt'  
time_series.x_range = Range1d(1979, 2022)
time_series.outline_line_color=None


time_series.scatter(source = source, x='year', y='Result', line_color = None, fill_alpha=0.5, color='green' )


#Create figure object for geomap.
geo = figure(title = "Vermont Lay monitoring programm",
           plot_height = 600,
           plot_width = 500,
           toolbar_location = 'left',
           tools = [hover, tap, 'pan', 'wheel_zoom'],
           x_axis_location=None,
           y_axis_location=None,
          )


geo.xgrid.visible = False
geo.ygrid.visible = False
geo.outline_line_color=None
# Add patch renderer to figure.
geo.patches("xs","ys",
          source = geosource,
          fill_color = {"field" :"Result",
                        "transform" : color_mapper},
          line_color = "black",
          line_width = 0.25,
          fill_alpha = 1)



# Define the callback function: update_plot
def update_plot(attr, old, new):
    
    yr = slider.value
    CharacteristicID = parameter_select.value
    
    new_data, color_mapper.low, color_mapper.high = json_data(yr, CharacteristicID)
    
    geosource.geojson = new_data
    geo.title.text = "Vermont Lay monitoring programm, {}\nParammeter: {}".format(yr,CharacteristicID)
    
    

# Define a callback function: update_time_series
def update_time_series(attr, old, new):
    
    selections = new
    CharacteristicID = parameter_select.value
    log = logging.getLogger('bokeh')
    log.info(selections)
    
    new_data = time_series_data(CharacteristicID, selections[0])
    source.data = new_data
    LakeID = time_series_LakeID(selections[0])
    log.info(LakeID)
    log.info(type(LakeID))
    time_series.title.text = "Lake {} {} data time-series".format(LakeID, CharacteristicID)
    
    
# Make a slider object: slider 
slider = Slider(title = "Year",start = 1980, end = 2021, step = 1, value = 1980, width=500)
slider.on_change("value",update_plot)

# Make a drop down object: selector 
LABELS = list(df.CharacteristicID.unique())
parameter_select = Select(value="TP", options=LABELS)
parameter_select.on_change("value", update_plot)
    
    
geosource.selected.on_change('indices', update_time_series)




#Specify layout
geo.add_layout(color_bar, "left")


first_column = column(geo, Column(slider, parameter_select))
second_column = column(time_series, plot, wedge)




# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = row(first_column, second_column)
curdoc().add_root(layout)